In [4]:
import pandas as pd
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch

# Step 1: Load and Inspect CSV
def load_and_inspect_csv(file_path):
    """Load a CSV, check for structural issues, and clean missing values."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    expected_columns = 6
    for i, line in enumerate(lines):
        columns = line.split(',')
        if len(columns) != expected_columns:
            print(f"Line {i + 1} has {len(columns)} columns: {line}")

    # Safely load the CSV while skipping bad lines
    df = pd.read_csv(file_path, on_bad_lines='skip')
    df.fillna('', inplace=True)
    return df

df = load_and_inspect_csv('151_ideas_updated.csv')

# Step 2: Initialize Models
def initialize_models():
    """Initialize the tokenizer, retriever, and model."""
    tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
    retriever = RagRetriever.from_pretrained(
        "facebook/rag-token-nq",
        use_dummy_dataset=True  # Use dummy dataset for testing
    )
    model = RagSequenceForGeneration.from_pretrained(
        "facebook/rag-token-nq",
        retriever=retriever
    ).half()
    return tokenizer, retriever, model

# Initialize models and move to the appropriate device
tokenizer, retriever, model = initialize_models()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Chain of Thought Function
def chain_of_thought(query, df):
    """Generate chain-of-thought prompts based on the input DataFrame."""
    steps = [
        f"Consider the idea: {row['Ideas']}. Themes: {row['Theme a']}, {row['Theme b']}, {row['Theme c']}" 
        for _, row in df.iterrows()
    ]
    return steps

# Step 4: Generate Response Function
def generate_response(query, model, tokenizer, batch_df):
    """Generate responses for a batch of ideas."""
    steps = chain_of_thought(query, batch_df)
    responses = []

    for step in steps:
        inputs = tokenizer([step], return_tensors="pt", padding=True, truncation=True).to(device)
        output = model.generate(**inputs, max_length=100)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        responses.append(response)

    return " ".join(responses)

# Step 5: REACT Function
def reason_over_actions(response, model, tokenizer):
    """Generate a REACT-style reasoning over actions."""
    prompt = f"Consider different outcomes for: {response}"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Step 6: Reflection Function
def reflect_and_improve(response, model, tokenizer):
    """Generate a reflection and improvement suggestion."""
    prompt = f"Reflect on this response: {response}. How can it be improved?"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Main Function
def main(query, df, model, tokenizer):
    """Run the entire pipeline in batches."""
    batch_size = 50
    final_responses = []

    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        initial_response = generate_response(query, model, tokenizer, batch_df)
        react_response = reason_over_actions(initial_response, model, tokenizer)
        final_response = reflect_and_improve(react_response, model, tokenizer)
        final_responses.append(final_response)

    return " ".join(final_responses)

# Example Query
query = "How can we combine these philosophical ideas effectively?"
final_output = main(query, df, model, tokenizer)
print(final_output)


Line 2 has 31 columns: "1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) ",fun,rational,positive,personal,"theme ideas - rough, rational, intuitive, positive , neutral, negative, fun, deep, chill, relational, inspiring, spicy, sexy, neech, opinion, philosophical, personal, stoic, deluze vibes, buddha, yoga boiii, emotion"

Line 3 has 11 columns: "2) Full Expression - it takes a lot of effort fo

/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this fu

OSError: Can't load 'psgs_w100.tsv.pkl'. Make sure that:

- 'https://storage.googleapis.com/huggingface-nlp/datasets/wiki_dpr/' is a correct remote path to a directory containing a file named psgs_w100.tsv.pkl

- or 'https://storage.googleapis.com/huggingface-nlp/datasets/wiki_dpr/' is the correct path to a directory containing a file named psgs_w100.tsv.pkl.



In [5]:
import pandas as pd
from transformers import RagTokenizer, RagSequenceForGeneration
import torch

# Step 1: Load and Inspect CSV
def load_and_inspect_csv(file_path):
    """Load CSV and inspect for invalid rows."""
    df = pd.read_csv(file_path, on_bad_lines='skip')
    df.fillna('', inplace=True)  # Replace NaN values with empty strings
    return df

df = load_and_inspect_csv('151_ideas_updated.csv')

# Step 2: Initialize Models
def initialize_model():
    """Initialize the tokenizer and generation model."""
    tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
    model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq").half()
    return tokenizer, model

tokenizer, model = initialize_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Chain of Thought Function
def chain_of_thought(query, df):
    """Create reasoning steps based on CSV ideas."""
    steps = [
        f"Consider the idea: {row['Ideas']}. Themes: {row['Theme a']}, {row['Theme b']}, {row['Theme c']}"
        for _, row in df.iterrows()
    ]
    return steps

# Step 4: Generate Response Function
def generate_response(query, model, tokenizer, batch_df):
    """Generate a response based on CSV ideas."""
    steps = chain_of_thought(query, batch_df)
    responses = []
    for step in steps:
        inputs = tokenizer([step], return_tensors="pt", truncation=True, padding=True).to(device)
        output = model.generate(**inputs)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        responses.append(response)
    return " ".join(responses)

# Step 5: Reflexion Function
def reflect_and_improve(response, model, tokenizer):
    """Refine the response through reflection."""
    prompt = f"Reflect on this response: {response}. How can it be improved?"
    inputs = tokenizer([prompt], return_tensors="pt", truncation=True, padding=True).to(device)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Main Function
def main(query, df, model, tokenizer):
    """Main processing loop."""
    batch_size = 50
    final_responses = []
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        initial_response = generate_response(query, model, tokenizer, batch_df)
        improved_response = reflect_and_improve(initial_response, model, tokenizer)
        final_responses.append(improved_response)
    return " ".join(final_responses)

# Example Query
query = "How can we combine these philosophical ideas effectively?"
final_output = main(query, df, model, tokenizer)
print(final_output)


/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this fu

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [9]:
import pandas as pd
from transformers import RagTokenizer, RagSequenceForGeneration
import torch

# Step 1: Load and Inspect CSV
def load_and_inspect_csv(file_path):
    """Load CSV and inspect for invalid rows."""
    df = pd.read_csv(file_path, on_bad_lines='skip')
    df.fillna('', inplace=True)  # Replace NaN values with empty strings
    return df

df = load_and_inspect_csv('151_ideas_updated.csv')

# Step 2: Initialize Models
def initialize_model():
    """Initialize the tokenizer and generation model."""
    tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
    model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq").half()
    return tokenizer, model

tokenizer, model = initialize_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Chain of Thought Function
def chain_of_thought(df):
    """Create reasoning steps based on CSV ideas."""
    steps = [f"Consider the idea: {row['Ideas']}" for _, row in df.iterrows()]
    return steps

# Step 4: Generate Response Function
def generate_response(step, model, tokenizer):
    """Generate a response based on a single reasoning step."""
    inputs = tokenizer([step], return_tensors="pt", truncation=True, padding=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Step 5: REACT Function
def reason_over_actions(response, model, tokenizer):
    """Generate a REACT-style reasoning over actions."""
    prompt = f"Consider different outcomes for: {response}"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Step 6: Reflection Function
def reflect_and_improve(response, model, tokenizer):
    """Generate a reflection and improvement suggestion."""
    prompt = f"Reflect on this response: {response}. How can it be improved?"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Main Function
def main(df, model, tokenizer):
    """Main processing loop."""
    steps = chain_of_thought(df)
    final_responses = []

    for step in steps:
        # Step 1: Generate Initial Response
        initial_response = generate_response(step, model, tokenizer)
        
        # Step 2: Perform REACT Reasoning
        react_response = reason_over_actions(initial_response, model, tokenizer)
        
        # Step 3: Reflect and Improve
        improved_response = reflect_and_improve(react_response, model, tokenizer)
        
        # Store the final improved response
        final_responses.append(improved_response)

    return "\n".join(final_responses)

# Example Execution
query = "How can we combine these philosophical ideas effectively?"
final_output = main(df, model, tokenizer)
print(final_output)


/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this fu

pytorch_model.bin:   6%|6         | 126M/2.06G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [10]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
import torch

# Step 1: Load and Inspect CSV
def load_and_inspect_csv(file_path):
    """Load CSV and inspect for invalid rows."""
    df = pd.read_csv(file_path, on_bad_lines='skip')
    df.fillna('', inplace=True)  # Replace NaN values with empty strings
    return df

df = load_and_inspect_csv('151_ideas_updated.csv')

# Step 2: Initialize Smaller Model
def initialize_model():
    """Initialize the tokenizer and generation model."""
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased").half()  # Ensure half precision
    return tokenizer, model

tokenizer, model = initialize_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Chain of Thought Function
def chain_of_thought(df):
    """Create reasoning steps based on CSV ideas."""
    steps = [f"Consider the idea: {row['Ideas']}" for _, row in df.iterrows()]
    return steps

# Step 4: Generate Response Function
def generate_response(step, model, tokenizer):
    """Generate a response based on a single reasoning step."""
    inputs = tokenizer([step], return_tensors="pt", truncation=True, padding=True).to(device)
    output = model(**inputs).logits
    response = torch.argmax(output, dim=-1).item()  # Example of a classification-based output
    return f"Generated response: {response}"

# Step 5: REACT Function
def reason_over_actions(response, model, tokenizer):
    """Generate a REACT-style reasoning over actions."""
    prompt = f"Consider different outcomes for: {response}"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model(**inputs).logits
    return f"Reasoned outcomes: {torch.argmax(output, dim=-1).item()}"

# Step 6: Reflection Function
def reflect_and_improve(response, model, tokenizer):
    """Generate a reflection and improvement suggestion."""
    prompt = f"Reflect on this response: {response}. How can it be improved?"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model(**inputs).logits
    return f"Reflected improvement: {torch.argmax(output, dim=-1).item()}"

# Main Function
def main(df, model, tokenizer):
    """Main processing loop."""
    steps = chain_of_thought(df)
    final_responses = []

    for step in steps:
        # Step 1: Generate Initial Response
        initial_response = generate_response(step, model, tokenizer)
        
        # Step 2: Perform REACT Reasoning
        react_response = reason_over_actions(initial_response, model, tokenizer)
        
        # Step 3: Reflect and Improve
        improved_response = reflect_and_improve(react_response, model, tokenizer)
        
        # Store the final improved response
        final_responses.append(improved_response)

    return "\n".join(final_responses)

# Example Execution
query = "How can we combine these philosophical ideas effectively?"
final_output = main(df, model, tokenizer)
print(final_output)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0
Reflected improvement: 0


In [3]:
# works but jank but alpha

# Analysis comparing responses at different stages
question = "What insights can we draw from these philosophical ideas?"  # Using single question for comparison

print(f"\nAnalyzing question across multiple stages: {question}")
steps = chain_of_thought(df)

# Compare 3 stages of processing for first 5 ideas
for step in steps[:5]:
    print(f"\n{'='*50}")
    print(f"Analyzing idea: {step[:100]}...")  # Print first 100 chars of idea
    
    # Stage 1 - Basic processing
    print("\nStage 1 - Basic Processing:")
    initial_response = generate_response(step, model, tokenizer)
    react_response = reason_over_actions(initial_response, model, tokenizer)
    first_reflection = reflect_and_improve(react_response, model, tokenizer)
    print(f"Result: {first_reflection}")
    
    # Stage 2 - Double reflection
    print("\nStage 2 - Double Reflection:")
    second_reflection = reflect_and_improve(first_reflection, model, tokenizer)
    print(f"Result: {second_reflection}")
    
    # Stage 3 - Triple reflection
    print("\nStage 3 - Triple Reflection:")
    third_reflection = reflect_and_improve(second_reflection, model, tokenizer)
    print(f"Result: {third_reflection}")

print("\nComparison Summary:")# Add this import at the top
from torch.nn import functional as F

# Modified analysis code
def generate_text_response(step, model, tokenizer, prefix=""):
    """Generate a text response instead of just classification"""
    prompt = f"{prefix}Analyze this philosophical idea: {step}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    
    # Get raw model output
    outputs = model(**inputs).logits
    
    # Create a simpler response based on the output value
    output_value = torch.argmax(outputs, dim=-1).item()
    
    # Map the output to a text response
    response_map = {
        0: "This idea focuses on practical wisdom and everyday application.",
        1: "This concept explores deeper metaphysical implications.",
        2: "This thought emphasizes personal growth and self-reflection."
    }
    
    return response_map.get(output_value, "This idea requires further analysis.")

# Main analysis code
question = "What insights can we draw from these philosophical ideas?"
print(f"\nAnalyzing question across multiple stages: {question}")
steps = chain_of_thought(df)

# Compare 3 stages of processing for first 5 ideas
for step in steps[:5]:
    print(f"\n{'='*50}")
    print(f"Analyzing idea: {step[:100]}...")
    
    # Stage 1 - Basic processing
    print("\nStage 1 - Basic Processing:")
    initial = generate_text_response(step, model, tokenizer)
    react = generate_text_response(step, model, tokenizer, "Consider implications of: ")
    reflection1 = generate_text_response(step, model, tokenizer, "Reflect deeply on: ")
    print(f"Initial: {initial}")
    print(f"REACT: {react}")
    print(f"Reflection: {reflection1}")
    
    # Stage 2 - Second round
    print("\nStage 2 - Deeper Analysis:")
    reflection2 = generate_text_response(step, model, tokenizer, "Building on previous analysis: ")
    print(f"Result: {reflection2}")
    
    # Stage 3 - Final synthesis
    print("\nStage 3 - Final Synthesis:")
    reflection3 = generate_text_response(step, model, tokenizer, "Synthesizing all analyses: ")
    print(f"Result: {reflection3}")

print("\nComparison Summary:")
print("Analyzing progression of insights across stages...")
print("Looking for patterns of improvement or degradation across stages...")


Analyzing question across multiple stages: What insights can we draw from these philosophical ideas?


NameError: name 'chain_of_thought' is not defined

In [ ]:
# older not working

import pandas as pd
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch

# Step 1: Load and Inspect CSV
def load_and_inspect_csv(file_path):
    """Load a CSV, check for structural issues, and clean missing values."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    expected_columns = 6
    for i, line in enumerate(lines):
        columns = line.split(',')
        if len(columns) != expected_columns:
            print(f"Line {i + 1} has {len(columns)} columns: {line}")

    # Safely load the CSV while skipping bad lines
    df = pd.read_csv(file_path, on_bad_lines='skip')
    df.fillna('', inplace=True)
    return df

df = load_and_inspect_csv('151_ideas_updated.csv')

# Step 2: Initialize Models
def initialize_models():
    """Initialize the tokenizer, retriever, and model."""
    tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
    retriever = RagRetriever.from_pretrained(
        "facebook/rag-token-nq",
        use_dummy_dataset=True  # Use dummy dataset for testing
    )
    model = RagSequenceForGeneration.from_pretrained(
        "facebook/rag-token-nq",
        retriever=retriever
    ).half()
    return tokenizer, retriever, model

# Initialize models and move to the appropriate device
tokenizer, retriever, model = initialize_models()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Chain of Thought Function
def chain_of_thought(query, df):
    """Generate chain-of-thought prompts based on the input DataFrame."""
    steps = [
        f"Consider the idea: {row['Ideas']}. Themes: {row['Theme a']}, {row['Theme b']}, {row['Theme c']}" 
        for _, row in df.iterrows()
    ]
    return steps

# Step 4: Generate Response Function
def generate_response(query, model, tokenizer, batch_df):
    """Generate responses for a batch of ideas."""
    steps = chain_of_thought(query, batch_df)
    responses = []

    for step in steps:
        inputs = tokenizer([step], return_tensors="pt", padding=True, truncation=True).to(device)
        output = model.generate(**inputs, max_length=100)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        responses.append(response)

    return " ".join(responses)

# Step 5: REACT Function
def reason_over_actions(response, model, tokenizer):
    """Generate a REACT-style reasoning over actions."""
    prompt = f"Consider different outcomes for: {response}"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Step 6: Reflection Function
def reflect_and_improve(response, model, tokenizer):
    """Generate a reflection and improvement suggestion."""
    prompt = f"Reflect on this response: {response}. How can it be improved?"
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Main Function
def main(query, df, model, tokenizer):
    """Run the entire pipeline in batches."""
    batch_size = 50
    final_responses = []

    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        initial_response = generate_response(query, model, tokenizer, batch_df)
        react_response = reason_over_actions(initial_response, model, tokenizer)
        final_response = reflect_and_improve(react_response, model, tokenizer)
        final_responses.append(final_response)

    return " ".join(final_responses)

# Example Query
query = "How can we combine these philosophical ideas effectively?"
final_output = main(query, df, model, tokenizer)
print(final_output)


Line 2 has 31 columns: "1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) ",fun,rational,positive,personal,"theme ideas - rough, rational, intuitive, positive , neutral, negative, fun, deep, chill, relational, inspiring, spicy, sexy, neech, opinion, philosophical, personal, stoic, deluze vibes, buddha, yoga boiii, emotion"

Line 3 has 11 columns: "2) Full Expression - it takes a lot of effort fo

/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this fu

OSError: Can't load 'psgs_w100.tsv.pkl'. Make sure that:

- 'https://storage.googleapis.com/huggingface-nlp/datasets/wiki_dpr/' is a correct remote path to a directory containing a file named psgs_w100.tsv.pkl

- or 'https://storage.googleapis.com/huggingface-nlp/datasets/wiki_dpr/' is the correct path to a directory containing a file named psgs_w100.tsv.pkl.

